### Part 1. SageMaker and Dask

In [11]:
import dask.dataframe as dd

In [21]:
from dask.distributed import Client

client = Client(n_workers=16)
client

Client Scheduler: tcp://127.0.0.1:45569 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 67.53 GB


In [22]:
%%time
comments = dd.read_csv('Youtube*.csv')

CPU times: user 10.4 ms, sys: 5.18 ms, total: 15.6 ms
Wall time: 14.3 ms


In [13]:
%%time
comments = comments.persist()

CPU times: user 49.4 ms, sys: 1.98 ms, total: 51.4 ms
Wall time: 40.5 ms


In [14]:
comments.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [15]:
comments['CLASS'].value_counts().compute()

1    1005
0     951
Name: CLASS, dtype: int64

In [38]:
#number of spam comments containing 'check' word
spam = comments[comments['CLASS'] == 1]
spam_check = spam[spam['CONTENT'].str.lower().str.contains('check')]

print("Number of word 'check' in spam:", len(spam_check))

Number of word 'check' in spam: 461


In [39]:
#number of legit comments containing 'check' word
legit = comments[comments['CLASS'] == 0]
legit_check = legit[legit['CONTENT'].str.lower().str.contains('check')]

print("Number of word 'check' in legit comments:", len(legit_check))

Number of word 'check' in legit comments: 19


### Part 2. Big Data Options

Both Spark and Desk are designed to scale from a single node to thousand nodes cluster. 
A data scientist wants to use them, when the data is too big for one single machine, which with
modern machines capabilities make up to 1 TB. Dask is built on Python and only supports Python,
while Spark is written in Scala with only some support fro Python. Generally, Dask is smaller and 
ligher weight than Spark, so it has fewer features, and ntended to be used with other libraries
particularly Python ones, like Numba. While Spark integrates well with many apache projects, and also
Spark has its own API and implements SQL Language. Dask has no SQL. 

Since Dask is lightweight and uses Python ecosystem, I'd use it when my project is already written in Python,
I need to scale it with minimum code rewriting. 
I'd use Spark for a larger set of features, when I prefer to use SQL, and query optimizer. 


In [ ]:

|              | Dask                                                        | Spark                                                      |
|--------------|-------------------------------------------------------------|------------------------------------------------------------|
| memory usage | lighter weight                                              | heavier weight                                             |
| language     | Python                                                      | Scala, Python                                              |
| ecosystem    | Python ecosystem                                            | All-in-one, integrates well  with many Apache projects     |
| age          | written in 2014                                             | written in 2010                                            |
| scope        | BI applications, scientific,  and other custom applications | Traditional BI, such as SQL and  lightweight ML            |
| scale        | From a single node to thousands                             | From a single node to thousands                            |
| APIs         | reuses the Pandas API                                       | DF API, it also implements subset  of SQL, query optimizer |